In [1]:
from __future__                           import print_function
%matplotlib inline

import re
import os
import sys
import h5py

import matplotlib.pyplot                  as plt
import numpy                              as np
import random                             as rd
import tables                             as tb
import tensorflow                         as tf

from glob                                 import glob
from matplotlib.patches                   import Ellipse


# Keras imports
import keras.backend.tensorflow_backend   as   K

from keras                                import regularizers
from keras                                import callbacks

from keras.models                         import Model, load_model

from keras.layers                         import Input, Dense, MaxPooling3D, AveragePooling3D
from keras.layers                         import Conv3D, Conv2D, AveragePooling2D, Activation, Dropout, ZeroPadding3D
from keras.layers                         import Add
from keras.layers                         import Flatten, BatchNormalization

from keras.layers.normalization           import BatchNormalization
from keras.layers.convolutional           import Convolution2D, MaxPooling2D
from keras.layers.core                    import Flatten

from keras.optimizers                     import SGD, Adam, Nadam, Adadelta 

from keras.regularizers                   import l2, l1
from keras.initializers                   import RandomNormal

from keras.utils.layer_utils              import print_summary


os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

Using TensorFlow backend.


### Create a combined file from several .npz files

In [ ]:
npz_dir = "/home/amenkara/PETALO_dnns/dnn/"

# input/output files

files = glob(npz_dir + '*.npz')

files = sorted(files)
print(files)

out_file = "combined_file_15_january.npz"

X_    = []
Compt_= []
Ev_   = []


for f in files:

    fn = np.load(f)
    
    if(len(X_) == 0):
        X_     = fn['images']
        Compt_ =  fn['labels']
        Ev_    =  fn['event_number']
    else:
        X_     = np.concatenate((X_,     fn['images'])      )
        Compt_ = np.concatenate((Compt_, fn['labels'])      ) 
        Ev_    = np.concatenate((Ev_,    fn['event_number']))
    
print("Saving combined file")
np.savez(out_file, X_=X_, Compt_=Compt_, Ev_=Ev_)

### Load the data from the dataset

In [2]:
out_file = "combined_file_15_january.npz"

In [3]:
data = np.load(out_file)
X_, Compt_, Ev_ = data['X_'],data['Compt_'], data['Ev_']

In [4]:
len(X_)

24561

### Separate train, validation and test set

In [5]:
train_cut = int(0.8*len(X_))
val_cut   = int(0.9*len(X_))

X_train      = X_[:train_cut]
X_val        = X_[train_cut:val_cut] 
X_test       = X_[val_cut:]

Compt_train  =  Compt_[:train_cut]
Compt_val    =  Compt_[train_cut:val_cut]
Compt_test   =  Compt_[val_cut:]

### Define the model

In [99]:
K.clear_session()

In [78]:
def model_CNN(inputs):
    initial = "random_uniform"
    #el 32 implica que hay 32 filtros de 5x5
    cinputs = Conv2D(96, (5, 5), padding='same', strides=(2, 2), activation='relu', kernel_initializer=initial, kernel_regularizer=regularizers.l1(0.01),  activity_regularizer=regularizers.l1(0.01))(inputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', kernel_initializer=initial, kernel_regularizer=regularizers.l1(0.01),  activity_regularizer=regularizers.l1(0.01))(cinputs)
    cinputs = Dropout(.2)(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    cinputs = Conv2D(384, (2, 2), padding='same', strides=(1, 1), activation='relu', kernel_initializer=initial, kernel_regularizer=regularizers.l1(0.01), activity_regularizer=regularizers.l1(0.01))(cinputs)
    cinputs = Dropout(.2)(cinputs)
    cinputs = BatchNormalization(epsilon=1e-05, axis=3, momentum=0.99, weights=None, beta_initializer='zero', gamma_initializer='one', gamma_regularizer=None, beta_regularizer=None)(cinputs)
    cinputs = AveragePooling2D(pool_size=(2, 2), data_format=None, padding="same", strides=(2, 2))(cinputs)
    
    f1 = Flatten()(cinputs)
    f1 = Dense(units=64, activation='relu', kernel_initializer=initial)(f1)
    #f1 = Dropout(.2)(f1)

    inc_output = Dense(units=1, activation='sigmoid', kernel_initializer=initial)(f1)
    
    model = Model(inputs, inc_output)
    
    model.compile(loss='binary_crossentropy',
                      optimizer=Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999,
                                      epsilon=1e-08, schedule_decay=0.001), metrics=['accuracy'])
    
    return model

In [100]:
def model_test(inputs):
    cinputs = Conv2D(96, (5, 5))(inputs)
    cinputs = Dense(32)(cinputs)
    internal = Dense(32)(cinputs)
    internal = Flatten()(internal)
    output = Dense(1, activation='sigmoid')(internal)
    
    model = Model(inputs, output)
    
    model.compile(loss='binary_crossentropy', optimizer=Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004))
    return model

In [101]:
dim     = 30

input_  = Input((dim,dim,1))
model   = model_test(input_)

In [102]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30, 30, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 96)        2496      
_________________________________________________________________
dense_1 (Dense)              (None, 26, 26, 32)        3104      
_________________________________________________________________
dense_2 (Dense)              (None, 26, 26, 32)        1056      
_________________________________________________________________
flatten_1 (Flatten)          (None, 21632)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21633     
Total params: 28,289
Trainable params: 28,289
Non-trainable params: 0
_________________________________________________________________


In [49]:
tboard = callbacks.TensorBoard(log_dir='./logs/combined_files_16_january_relu', histogram_freq=0, write_graph=True, write_images=False)

### Training

In [103]:
model.fit(X_train, Compt_train, batch_size=18, epochs=1, shuffle=True, callbacks=[tboard],verbose=1, validation_data=(X_val, Compt_val))

Train on 19648 samples, validate on 2456 samples
Epoch 1/1
19648/19648 [==============================] - 15s 772us/step - loss: 0.6209 - val_loss: 0.6158


In [11]:
model.save('model_conv2d_16_january_last_layer_activation_is_relu.h5')

In [26]:
loaded_model = load_model('model_conv2d_16_january.h5')

In [104]:
prediction = model.predict(X_test, batch_size=18, verbose=1)

2457/2457 [==============================] - 1s 291us/step


In [105]:
print(prediction[0:300])

[[0.6651584 ]
 [0.61720735]
 [0.66559213]
 [0.6395261 ]
 [0.66168976]
 [0.6297166 ]
 [0.6659271 ]
 [0.66038036]
 [0.6620611 ]
 [0.66171694]
 [0.6880801 ]
 [0.6741651 ]
 [0.6608642 ]
 [0.6634508 ]
 [0.66951734]
 [0.62039405]
 [0.663626  ]
 [0.66710883]
 [0.6698348 ]
 [0.6378188 ]
 [0.6668746 ]
 [0.6646208 ]
 [0.683938  ]
 [0.65236354]
 [0.66071075]
 [0.65450114]
 [0.66001374]
 [0.6261461 ]
 [0.65842605]
 [0.6332646 ]
 [0.63533473]
 [0.6598323 ]
 [0.6556044 ]
 [0.65787894]
 [0.6567627 ]
 [0.6401634 ]
 [0.68798864]
 [0.65681374]
 [0.6450932 ]
 [0.66518235]
 [0.66268796]
 [0.62239456]
 [0.6508017 ]
 [0.65789723]
 [0.6654121 ]
 [0.6562228 ]
 [0.66111267]
 [0.68263507]
 [0.64777625]
 [0.6600907 ]
 [0.66183233]
 [0.64968926]
 [0.6691289 ]
 [0.6635079 ]
 [0.6716137 ]
 [0.64486307]
 [0.63430965]
 [0.649767  ]
 [0.6539066 ]
 [0.62046796]
 [0.6641039 ]
 [0.651376  ]
 [0.6767648 ]
 [0.66293347]
 [0.6506183 ]
 [0.6393618 ]
 [0.6509482 ]
 [0.66011363]
 [0.6611381 ]
 [0.66840756]
 [0.65233064]
 [0.67

In [ ]:
#Cut is the accuracy. 
def plot_result(cut):
elements = []
for element in prediction:
    if element > cut:
        elements.append(0)
    else:
        elements.append(1)
        
    aciertos = 0
    total = len(elements)
    for i in range(total):
        if elements[i]== Compt_test[i]:
            aciertos += 1
        
    print(aciertos)    
    return aciertos/total

In [ ]:
matrix = []
for i in range(500, 1000):
    matrix.append([i/1000., plot_result(i/1000.)])
        

In [ ]:
np.transpose(matrix)[1]

In [ ]:
plt.plot(np.transpose(matrix)[0][:], np.transpose(matrix)[1][:])